# Training Dataset 

The training dataset is in dataset_train.csv file. 

- [Preview](#preview)
    - [Reading data to pandas dataframe](#reading-data-to-pandas-dataframe)
    - [Describing dataset with statistics](#describing-dataset-with-statistics)
    - [Plots](#plots)
        - [Histogram](#histogram)
        - [Boxplot](#boxplot)
        - [Scatter plot](#scatter-plot)
        - [Pair Plot Matrix](#pair-plot-matrix)
        - [Boxplot Matrix](#boxplot-matrix)
        - [Conclusion : selecting features](#conclusion--selecting-features)
    - [Conclusion : selecting features](#conclusion--selecting-features)

- [Data preparation](#data-preparation)
    - [Selecting features](#selecting-features)
    - [Drop NaN containing rows](#drop-nan-containing-rows)
    - [Standardization : z-score method](#standardization)
    - [Data quality check](#data-quality)
        - [Boxplot matrix standardized](#boxplot-matrix-standardized)
        - [Heatmap](#heatmap-on-training-dataset)

- [Exploring other features](#exploring-other-features)
        - [Best Hand](#best-hand)


## Preview

Reading the csv file and looking at dataset.

### Reading data to pandas dataframe

In [ ]:
# %pip install seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dataset_train = f'../datasets/dataset_train.csv'
df = pd.read_csv(dataset_train)
df.head()

The `Hogwarts house` is the real outcome. We need to train our model with selected features and so that the predicted output `house` could later be compared to the real one. 
We need to inspect the features:
- non numeric or biographic features (first and last names, birthday dates and best hand) might be enriching our model accuracy but are harder to sort out. we will inspect them in a second round of training
- numeric values for columns Arithmancy to Flying, which represent 13 features is a good start

First, we will be focusing on `features` with `numeric values`.
Selecting numeric columns, where dtypes are np.number
There is total of `13 features` that has `numeric types`, index is not counted.
Standardization and handling missing data (NaN) will be required.

In [ ]:
df.select_dtypes(include=np.number).head(10)

### Describing dataset with statistics

To describe statitics to dataset, we could have used pandas module describe() function
```python
df[df.columns[1:]].describe()
```
However, `pd.describe()` is a forbidden function and using it would be considered as cheating.

We will use our own `describe.py` Python program launched from `jupyter notebook`.
For that purpose, we can import the `os module`, to interact with the operating system
and execute it, of course in our `virtual environment`

In [ ]:
import os

script_path = '../dslr/describe.py'
dataset_train = f'../datasets/dataset_train.csv'
os.system(f'../venv/bin/python {script_path} {dataset_train}')


Missing values for numeric columns features 

In [ ]:
df[df.select_dtypes(include=np.number).columns[1:]].isna().sum()

In [ ]:
df[df.select_dtypes(include=np.number).columns[1:]].notna().sum()

### Plots

Exploring these 13 features.

    Only the meaningful variables should be included.
    The independent variables should be independent of each other. 
    That is, the model should have little or no multicollinearity.

In [ ]:
df[df.select_dtypes(include=np.number).columns[1:]].columns.to_list()

Plots can be of 3 kinds :
- `distibution`, such as histograms
- `categorical`, such as boxplots
- `relational` such as scatter plot

Using inline matplotlib module. (Affichage avec la bibliothèque graphique intégrée à Notebook)



#### Histogram
Distribution of a given features among Hogwarts houses.
It looks like 2 houses (Griffindor and Slytherin) are not good at `Herbology`.
`Herbology` features might be a good feature for our model beacuse it might allow a better classification.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df_herb = df.groupby('Hogwarts House')['Herbology']
df_herb.plot(kind='hist', alpha=0.4, legend=True)
plt.show()

#### Boxplot

A `boxplot` is a graphical and standardised way to display the distribution of data based on five key numbers:

- “minimum”
- 1st Quartile (25th percentile)
- median (2nd Quartile/ 50th Percentile)
- 3rd Quartile (75th percentile)
- “maximum”

In [ ]:
import seaborn as sns

sns.boxplot(x='Hogwarts House', y='Astronomy', data=df)

#### Scatter plot

In [ ]:
plt.scatter('Astronomy', 'Herbology', marker='.', alpha=0.3, data=df)
plt.show()

In [ ]:
sns.scatterplot(
 data=df, 
 x="Astronomy",
 y="Herbology",
 hue="Hogwarts House",
)

In [ ]:
sns.scatterplot(
 data=df, 
 x="Astronomy",
 y="Defense Against the Dark Arts",
 hue="Hogwarts House",
 legend='auto'
)

#### Pair Plot Matrix

Lower triangle matrix with `Pair plots`.

In [ ]:
target="Hogwarts House"
remove_list = ['Index', 'First Name', 'Last Name', 'Birthday', 'Best Hand']

df_num = df.drop(remove_list, axis=1)
features = df_num.keys()[1:].to_list()
sns.pairplot(df_num,
                x_vars=features,
                y_vars=features,
                hue=target,
                corner=True
            )

#### Boxplot Matrix

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(20, 20))
features = df[df.select_dtypes(include=np.number).columns[1:]].columns.to_list()
for idx in range(13):
    i = idx // 4
    j = idx % 4
    sns.boxplot(data=df, x="Hogwarts House", ax=axs[i, j], y=features[idx])
plt.show()

In [ ]:
sns.boxplot(data=df, x="Hogwarts House", y="Care of Magical Creatures")

### Conclusion : selecting features

Two features['Arithmancy', 'Care of Magical Creatures'] cannot classify well, and "Defense Against the Dark Arts" is anti-correlated with  "Astronomy".

These features will be dropped before model training.

## Data preparation

Before training, we need to
- Keep only `meaningful features`
- Remove rows containg `NaN`
- `Standardize` input data

### Selecting features

Selecting `Meaningful` and `independent` features.

- pd.drop() : down to to 10 meaningful features, independent from each other

Thus, we can start training our model with `10 numeric features`

- [ ] 'Arithmancy',
- [X] 'Astronomy',
- [X] 'Herbology',
- [ ] 'Defense Against the Dark Arts',
- [X] 'Divination',
- [X] 'Muggle Studies',
- [X] 'Ancient Runes',
- [X] 'History of Magic',
- [X] 'Transfiguration',
- [X] 'Potions',
- [ ] 'Care of Magical Creatures',
- [X] 'Charms',
- [X] 'Flying'

### Drop NaN containing rows

- pd.dropna() : Dropping rows that contain NaN => down to 1333 rows

In [ ]:
df_train= df.drop(df.columns[2:6], axis = 1)
excluded_features = ["Index", "Arithmancy", "Defense Against the Dark Arts", "Care of Magical Creatures"]
df_train.drop(excluded_features, inplace=True, axis=1)
df_train.dropna(inplace=True)
df_train.head()

### Standardization

Values are very different between features, they need to be scaled.

`Standardizing input values` for each `feature`, applied along axis=1, using the `z-score method`.

The `z-score` method (often called standardization) transforms the input into distribution with a `mean of 0`` and a `standard deviation of 1``.

Each standardized value is computed by subtracting the mean of the corresponding feature then dividing by the quality deviation.

In [ ]:
def standardize(arr: pd.Series):
    """z-score method, using pandas std"""
    mean = arr.mean()
    std = arr.std()
    return (arr - mean) / std

df_train.iloc[:,1:] = df_train.iloc[:,1:].agg(lambda feature: standardize(feature))
df_train.head()

### Data quality 

#### Boxplot matrix standardized

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 5, figsize=(20, 20))
features = df_train.columns[1:].to_list()
for idx in range(10):
    i = idx // 5
    j = idx % 5
    sns.boxplot(data=df_train, x='Hogwarts House', ax=axs[i, j], y=features[idx])
plt.show()

#### Heatmap on training dataset

In [ ]:
df_heat = df_train.groupby(["Hogwarts House"]).mean()
sns.heatmap(df_heat, cmap='YlGnBu')

### Exploring other features

#### Best Hand

`Best Hand` feature is binomial, either 'Left" or 'Right', there is no ambidextrious wizards.

In [ ]:
df['Best Hand'].unique()

Is `Best Hand` significative for house determination ?


In [ ]:
def standardize(arr: pd.Series):
    """z-score method, using pandas std"""
    mean = arr.mean()
    std = arr.std()
    return (arr - mean) / std

df3 = df.drop(df.columns[2:5], axis = 1)
df3.iloc[:,3:] = df3.iloc[:,3:].agg(lambda feature: standardize(feature))
df3.head()

In [ ]:
df3 = df3.drop(df.columns[0], axis = 1).groupby(["Hogwarts House", "Best Hand"]).mean()
df3.head(8)

In [ ]:
sns.heatmap(df3, cmap='YlGnBu')